In [1]:
import pandas as pd

In [2]:
def established_profitable_companies(
    df,
    max_market_rank=500,
    top_roa_threshold_pc=0.25,
    top_roe_threshold_pc=0.25,
    bottom_debt_to_equity_ratio=0.25,
) -> pd.DataFrame:
    """Focus on Established, Profitable Companies

    **Criteria:**

    *   **Market Capitalization**: Preferably large-cap stocks.
    *   **Profitability Ratios**: Positive and stable profit margins, operating margins, and gross margins.
    *   **Return Ratios**: High return on assets (ROA) and return on equity (ROE).
    *   **Debt Levels**: Low debt-to-equity ratio.
    *   **Dividend History**: Regular and stable dividends.

    Args:
        df (_type_): _description_
        max_market_rank (int, optional): _description_. Defaults to 500.
        top_roa_threshold_pc (float, optional): _description_. Defaults to 0.25.
        top_roe_threshold_pc (float, optional): _description_. Defaults to 0.25.
        bottom_debt_to_equity_ratio (float, optional): _description_. Defaults to 0.25.

    Returns:
        pd.DataFrame: Recommnded stocks
    """
    # Market Capitalization: Preferably large-cap stocks (manual segmentation needed)

    df_large_cap = df.query(f"market_cap_rank < {max_market_rank}")

    # Profitability Ratios: Positive and stable profit margins, operating margins, and gross margins
    df_profitable = df_large_cap.query(
        "profit_margins > 0 and operating_margins > 0 and gross_margins > 0"
    )

    # Return Ratios: High return on assets (ROA) and return on equity (ROE)
    # Top top_roa_threshold_pc% ROA and Top top_roe_threshold_pc% ROE
    roa_threshold = df_profitable["return_on_assets"].quantile(1 - top_roa_threshold_pc)
    roe_threshold = df_profitable["return_on_equity"].quantile(1 - top_roe_threshold_pc)

    df_high_return = df_profitable.query(
        f"return_on_assets > {roa_threshold} and return_on_equity > {roe_threshold}"
    )

    # Debt Levels: Low debt-to-equity ratio
    # Bottom bottom_debt_to_equity_ratio % debt-to-equity
    debt_to_equity_threshold = df_high_return["debt_to_equity"].quantile(
        bottom_debt_to_equity_ratio
    )

    df_low_debt = df_high_return.query(f"debt_to_equity < {debt_to_equity_threshold}")

    # Dividend History: Regular and stable dividends
    df_dividends = df_low_debt.dropna(
        subset=["ex_dividend_date", "last_dividend_value"]
    )

    # Final filtered dataframe
    df_filtered = df_dividends

    return df_filtered[
        [
            "symbol",
            "name_of_company",
            "day_high",
            "market_cap",
            "profit_margins",
            "operating_margins",
            "gross_margins",
            "return_on_assets",
            "return_on_equity",
            "debt_to_equity",
            "ex_dividend_date",
            "last_dividend_value",
        ]
    ]


def strong_financial_health_and_linquidity(
    df,
    min_quick_ratio=1,
    min_current_ratio=1,
    min_cash_to_debt_qn=0.8,
    min_free_cashflow=0,
    min_operating_cashflow=0,
):
    # Filtering criteria for strong financial health and liquidity

    # Quick Ratio: Greater than 1
    df_quick_ratio = df.query(f"quick_ratio > {min_quick_ratio}")

    # Current Ratio: Greater than 1
    df_current_ratio = df_quick_ratio.query(f"current_ratio > {min_current_ratio}")

    # Total Cash and Cash Per Share: High values relative to total debt

    cash_to_debt_threshold = (
        df_current_ratio["total_cash"] / df_current_ratio["total_debt"]
    ).quantile(min_cash_to_debt_qn)

    df_cash = df_current_ratio[
        (df_current_ratio["total_cash"] / df_current_ratio["total_debt"])
        > cash_to_debt_threshold
    ]

    # Free Cashflow and Operating Cashflow: Positive and growing
    df_cashflow = df_cash.query(
        f"free_cashflow > {min_free_cashflow} and operating_cashflow > {min_operating_cashflow}"
    )

    # Assuming 'previous_free_cashflow' and 'previous_operating_cashflow' columns exist for growth calculation
    # Uncomment the following lines if historical data is available for comparison
    # df_cashflow_growth = df_cashflow[
    #     (df_cashflow['free_cashflow'] > df_cashflow['previous_free_cashflow']) &
    #     (df_cashflow['operating_cashflow'] > df_cashflow['previous_operating_cashflow'])
    # ]

    # Display the filtered stocks
    return df_cashflow[
        [
            "symbol",
            "name_of_company",
            "day_high",
            "quick_ratio",
            "current_ratio",
            "total_cash",
            "total_cash_per_share",
            "total_debt",
            "free_cashflow",
            "operating_cashflow",
            "recommendation_key",
        ]
    ].sort_values(by=["day_high"])

In [3]:
df = pd.read_csv("../data/base/all_stocks.csv")

In [4]:
established_profitable_companies(df, max_market_rank=1000)

,symbol,name_of_company,day_high,market_cap,profit_margins,operating_margins,gross_margins,return_on_assets,return_on_equity,debt_to_equity,ex_dividend_date,last_dividend_value
29,TRITURBINE,Triveni Turbine Limited,646.80,20185.189581,0.16273,0.18457,0.50470,0.12445,0.31288,0.325,1.708042e+09,1.00
59,ESABINDIA,Esab India Limited,6171.90,9389.114163,0.13108,0.17611,0.40159,0.26876,0.57557,1.493,1.722470e+09,30.00
84,BLS,BLS International Services Limited,381.00,15606.125363,0.18664,0.17694,0.30055,0.15356,0.28579,2.122,1.708646e+09,0.50
87,OFSS,Oracle Financial Services Software Limited,10488.00,90400.954778,0.34825,0.43629,0.52913,0.17621,0.28978,0.547,1.715040e+09,240.00
111,ZENTEC,Zen Technologies Limited,1454.00,11781.781094,0.29074,0.33591,0.70743,0.17471,0.32515,1.294,1.694131e+09,0.20
151,ASTRAZEN,AstraZeneca Pharma India Limited,7190.00,17239.374234,0.12474,0.11906,0.54107,0.10271,0.24851,0.646,1.720138e+09,24.00
164,SKFINDIA,SKF India Limited,6460.00,31590.141133,0.12074,0.16161,0.39208,0.11769,0.21966,0.438,1.720051e+09,130.00
216,SUPREMEIND,Supreme Industries Limited,6109.95,76721.133978,0.10555,0.14152,0.31824,0.12869,0.22494,1.078,1.718928e+09,22.00
223,3MINDIA,3M India Limited,39449.95,43672.653005,0.13926,0.18398,0.40153,0.14803,0.30510,0.847,1.720138e+09,525.00
237,CGPOWER,CG Power and Industrial Solutions Limited,778.90,117982.887936,0.17736,0.11928,0.30438,0.12545,0.36220,0.582,1.707091e+09,1.30


In [5]:
strong_financial_health_and_linquidity(pd.read_csv("../data/base/all_stocks.csv"))

,symbol,name_of_company,day_high,quick_ratio,current_ratio,total_cash,total_cash_per_share,total_debt,free_cashflow,operating_cashflow,recommendation_key
718,SOMATEX,Soma Textiles & Industries Limited,34.74,1.716,1.831,3.503000e+08,10.629,16200000.0,1.391570e+08,2.593000e+08,none
1270,ACEINTEG,Ace Integrated Solutions Limited,34.87,7.054,12.270,3.230000e+07,3.841,2000000.0,8.262500e+06,3.700000e+06,none
178,DIGIDRIVE,Digidrive Distributors Limited,36.85,3.743,4.748,1.810000e+08,4.684,900000.0,6.610000e+07,7.630000e+07,none
1012,NIRAJ,Niraj Cement Structurals Limited,53.49,1.118,2.382,2.294390e+08,5.719,7242000.0,6.297372e+08,2.691250e+08,none
655,DEN,Den Networks Limited,55.80,7.408,7.644,2.930769e+10,61.536,255110000.0,2.108471e+09,8.396700e+08,none
...,...,...,...,...,...,...,...,...,...,...,...
1729,VOLTAMP,Voltamp Transformers Limited,13468.90,2.107,3.709,6.060010e+08,59.899,32143000.0,1.600190e+09,2.162775e+09,buy
721,ABBOTINDIA,Abbott India Limited,27997.15,1.827,2.417,2.134490e+10,1004.466,832300032.0,1.020105e+10,1.212810e+10,none
822,BOSCHLTD,Bosch Limited,35680.00,1.094,1.948,3.389800e+10,1149.286,393000000.0,7.402625e+09,1.252800e+10,none
223,3MINDIA,3M India Limited,39449.95,1.945,2.519,1.357843e+10,1205.344,181778000.0,5.623520e+09,6.430349e+09,none
